In [1]:
from kedmi.models.generator import *
from kedmi.models.discri import *
from kedmi.utils.helper import *
import torch
import os
import numpy as np
from kedmi.utils.kedmi_attack import mnist_inversion, dist_inversion

### load specified configuration and specify environment variables

In [2]:
CONFIG_PATH = "config/kedmi/config/celeba/attacking/mnist_cust.json"
configuration = load_json(json_file=CONFIG_PATH)

save_folder = os.path.join(
    f"{configuration['dataset']['name']}_{configuration['dataset']['model_name']}",
    configuration["attack"]["variant"],
)
prefix = os.path.join(
    os.path.join(configuration["root_path"], "kedmi_300ids"), save_folder
)
save_dir = os.path.join(prefix, "latent")
save_img_dir = os.path.join(
    prefix, "imgs_{}".format(configuration["attack"]["variant"])
)

os.makedirs(prefix, exist_ok=True)
os.makedirs(save_img_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(configuration["dataset"]["p_reg_path"], exist_ok=True)

### load trained models

In [3]:
targetnets, E, G, D, n_classes, fea_mean, fea_logvar = get_attack_model(configuration)

['VGG16', 'VGG16']
Load MNIST_GAN
path_G checkpoints/gan/Generatormnist.tar
path_D checkpoints/gan/Discriminatormnist.tar
0
VGG16
Load classifier VGG16 at checkpoints/VGG16/nn_VGG16mnist.tar


/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1
VGG16
Load classifier VGG16 at checkpoints/VGG16/nn_VGG16mnist.tar


### set necessary params

In [4]:
print(D)

DataParallel(
  (module): DGWGAN_MNIST(
    (ls): Sequential(
      (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): LeakyReLU(negative_slope=0.2)
      (2): Sequential(
        (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
      (3): Sequential(
        (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
      (4): Sequential(
        (0): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
      (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
  

In [5]:
N = 5
bs = 60

### Run attack

In [6]:
# Begin attacking
for i in range(1):
    iden = torch.from_numpy(np.arange(bs))

    # evaluate on the first 300 identities only
    target_cosines = 0
    eval_cosines = 0
    for idx in range(5):
        iden = iden % n_classes
        print(
            "--------------------- Attack batch [%s]------------------------------"
            % idx
        )
        print("Iden:{}".format(iden))
        save_dir_z = "{}/{}_{}".format(save_dir, i, idx)
        print("kedmi")

        mnist_inversion(
            G,
            D,
            targetnets,
            E,
            iden,
            lr=configuration["attack"]["lr"],
            iter_times=configuration["attack"]["iters_mi"],
            momentum=0.9,
            lamda=100,
            clip_range=1,
            improved=configuration['attack']['improved_flag'],
            num_seeds=configuration['attack']['num_seeds'],
            used_loss=configuration['attack']['loss'],
            prefix=save_dir_z,
            save_img_dir=os.path.join(save_img_dir, "{}_".format(idx)),
            fea_mean=fea_mean,
            fea_logvar=fea_logvar,
            lam=configuration["attack"]["lam"],
            clipz=configuration['attack']['clipz'],
        )
        iden = iden + bs

--------------------- Attack batch [0]------------------------------
Iden:tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
        4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
        8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
kedmi
criterion:logit_loss


/home/bot/.local/lib/python3.10/site-packages/torch/nn/parallel/comm.py:227: UserWarning: Using -1 to represent CPU tensor is deprecated. Please use a device object or string instead, e.g., "cpu".
  warnings.warn(


torch.Size([60, 1, 64, 64])
Iteration:300	Prior Loss:24.60	Iden Loss:-15.42	Attack Acc:93.33
torch.Size([60, 1, 64, 64])
Iteration:600	Prior Loss:23.96	Iden Loss:-17.87	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:900	Prior Loss:23.91	Iden Loss:-18.51	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:1200	Prior Loss:23.33	Iden Loss:-19.17	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:1500	Prior Loss:23.47	Iden Loss:-19.66	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:1800	Prior Loss:23.17	Iden Loss:-20.20	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:2100	Prior Loss:22.97	Iden Loss:-20.56	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:2400	Prior Loss:22.47	Iden Loss:-21.13	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:2700	Prior Loss:22.48	Iden Loss:-21.52	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:3000	Prior Loss:22.69	Iden Loss:-22.02	Attack Acc:100.00
torch.Size([60, 1, 64, 64])
Iteration:3300	Prior Loss:

KeyboardInterrupt: 